References
#https://www.kaggle.com/edumunozsala/feature-eng-and-a-simple-stacked-model
#https://www.kaggle.com/nadare/eda-feature-engineering-and-modeling-4th-359

#https://www.kaggle.com/arateris/2-layer-k-fold-learning-forest-cover

In [1]:
# For the feature engineer, there are something needed to be checked for example, if there is some value missing?
# The distribution for the data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc


from sklearn.neighbors import KNeighborsClassifier

from time import time

from collections import Counter
from itertools import combinations

from sklearn.model_selection import cross_val_score,cross_validate, cross_val_predict,train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from mlxtend.classifier import StackingCVClassifier, StackingClassifier
# import lightgbm
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


from tqdm import tqdm

from scipy.stats import gaussian_kde

from datetime import datetime


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
#import the data
train_path='/home/ubuntu/learn-with-other-kaggle/data/train.csv'
test_path='/home/ubuntu/learn-with-other-kaggle/data/test.csv'

train_data=pd.read_csv(train_path, index_col = 'Id')
test_data=pd.read_csv(test_path, index_col = 'Id')

X = train_data.drop('Cover_Type', axis=1)
y = train_data['Cover_Type']

In [3]:
tune_ = False
# no missing data
print(train_data.isnull().any())
print(test_data.isnull().any())

Elevation                             False
Aspect                                False
Slope                                 False
Horizontal_Distance_To_Hydrology      False
Vertical_Distance_To_Hydrology        False
Horizontal_Distance_To_Roadways       False
Hillshade_9am                         False
Hillshade_Noon                        False
Hillshade_3pm                         False
Horizontal_Distance_To_Fire_Points    False
Wilderness_Area1                      False
Wilderness_Area2                      False
Wilderness_Area3                      False
Wilderness_Area4                      False
Soil_Type1                            False
Soil_Type2                            False
Soil_Type3                            False
Soil_Type4                            False
Soil_Type5                            False
Soil_Type6                            False
Soil_Type7                            False
Soil_Type8                            False
Soil_Type9                      

In [4]:
# the rfc used for some feature dection
rfc = RandomForestClassifier(n_estimators=100)

In [5]:
# the function used to check the feature importance
def importance_check(clf, X, y):

    cols_ = X.columns.values
    clf.fit(X,y)
    imp_df = pd.DataFrame({"feature":cols_,"imp":clf.feature_importances_})
    imp_df.sort_values(by='imp',ascending=False,inplace=True)
    plt.figure(figsize=(15,12))
    plt.xticks(rotation='vertical')
    sns.barplot(y=imp_df['feature'], x=imp_df['imp'])
    return imp_df

In [6]:
# the cv check for the model
def cv_check(clf, X,y,cv=5,clf_name=None):

    start_ = datetime.now()
    scores = cross_val_score(clf,X,y, cv=cv, n_jobs = -1, verbose=1)
    end_ = datetime.now()
    if clf_name is None:
        print("mean %f, std %f, time taken %s"%(scores.mean(),scores.std(), (end_-start_)))
    else:
        print("classifer %s, mean %f, std %f, time taken %s"%(clf_name, scores.mean(),scores.std(), (end_-start_)))
    return scores.mean(), scores.std

In [7]:
def confusion_matrix_check(clf, X,y,fig_w = 10, fig_h = 8):

    pred = cross_val_predict(clf, X,y,cv = 3, n_jobs = -1)
    conf_max = confusion_matrix(y,pred)
    classes = sorted(y.unique().tolist())
    
    plt.figure(figsize=(fig_w,fig_h))
    ax = sns.heatmap(conf_max,cmap='BuPu',annot=True,fmt='d',xticklabels = classes, yticklabels=classes)
    ax.set(xlabel='Predicted Class', ylabel='True Class', title='Confuse Matrix')
    ax.set_ylim(7,0)

In [8]:
def show_feature_correl(X, num_cols = None, w=10, h=10):

    if num_cols is not None:
        numericals = num_cols
    else:    
        numericals = [col for col in X.columns if 
                  'Soil_Type' not in col and 
                  'Wilderness_Area' not in col]
        numericals = sorted(numericals)
    plt.figure(figsize=(w,h))
    ax = sns.heatmap(X[numericals].corr().abs(),annot=True, linewidths=1, square=True, 
                cmap='BuPu', cbar=False)
    high_,low_=ax.get_ylim()
    ax.set_ylim(high_+0.5,low_-0.5)

In [9]:
def drop_unimportance_features(X_, threshold=0.99):
    
    feat_list = X_.columns
    not_import_feature = []
    
    for f in feat_list:
        c = X_[f].value_counts().iat[0]/X_.shape[0]
        
        if c > threshold:
            not_import_feature.append(f)
    result = X_.drop(not_import_feature, axis=1)
    return not_import_feature, result

In [10]:
def drop_unimportance_features_rate(clf, X, y, threshold=0.01):
    
    cols_ = X.columns.values
    clf.fit(X,y)
    imp_df = pd.DataFrame({"feature":cols_,"imp":clf.feature_importances_})
    result = X[imp_df[imp_df['imp']>=threshold]["feature"]]
    return result

In [11]:
def save_result(clf, test_data, fn=None):
    
    if fn is None:
        fn = datetime.now().strftime("%Y-%m-%d-%H-%M")+".csv"
    pred = clf.predict(test_data)
    final_result = pd.DataFrame({'Id':test_data.index, 'Cover_Type':pred})
    final_result.to_csv(fn, index=False)

In [12]:
def kde_check(X,y, features):
    _ , axs= plt.subplots(1,len(features), figsize=(5*len(features),5))
    #axs = axs.toList()
    for i, f in enumerate(features):
        for j in y.unique():
            sns.kdeplot(X_add_fea[y[:]==j][f], label=j, ax=axs[i])

In [13]:
# check the basic score and importance for all the features
if tune_:
    importance_check(rfc, X, y)
    cv_check(rfc, X, y, cv = 5)

In [14]:
# feature engineer to do
def add_features(X_):
    X = X_.copy()
    
    X['Hydro_Elevation_sum'] = X[['Elevation',
                                  'Vertical_Distance_To_Hydrology']
                                 ].sum(axis='columns')
    
    X['Hydro_Elevation_diff'] = X[['Elevation',
                                   'Vertical_Distance_To_Hydrology']
                                  ].diff(axis='columns').iloc[:, [1]]

    X['Hydro_Euclidean'] = np.sqrt(X['Horizontal_Distance_To_Hydrology']**2 +
                                   X['Vertical_Distance_To_Hydrology']**2)

    X['Hydro_Manhattan'] = (X['Horizontal_Distance_To_Hydrology'] +
                            X['Vertical_Distance_To_Hydrology'].abs())
    
    
    X['Hydro_Distance_sum'] = X[['Horizontal_Distance_To_Hydrology',
                                 'Vertical_Distance_To_Hydrology']
                                ].sum(axis='columns')

    X['Hydro_Distance_diff'] = X[['Horizontal_Distance_To_Hydrology',
                                  'Vertical_Distance_To_Hydrology']
                                 ].diff(axis='columns').iloc[:, [1]]
    
    X['Hydro_Fire_sum'] = X[['Horizontal_Distance_To_Hydrology',
                             'Horizontal_Distance_To_Fire_Points']
                            ].sum(axis='columns')

    X['Hydro_Fire_diff'] = X[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Fire_Points']
                             ].diff(axis='columns').iloc[:, [1]].abs()

    X['Hydro_Fire_mean'] = X[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Fire_Points']
                             ].mean(axis='columns')

    X['Hydro_Fire_median'] = X[['Horizontal_Distance_To_Hydrology',
                                'Horizontal_Distance_To_Fire_Points']
                               ].median(axis='columns')
                               
    X['Hydro_Road_sum'] = X[['Horizontal_Distance_To_Hydrology',
                             'Horizontal_Distance_To_Roadways']
                            ].sum(axis='columns')

    X['Hydro_Road_diff'] = X[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Roadways']
                             ].diff(axis='columns').iloc[:, [1]].abs()

    X['Hydro_Road_mean'] = X[['Horizontal_Distance_To_Hydrology',
                              'Horizontal_Distance_To_Roadways']
                             ].mean(axis='columns')

    X['Hydro_Road_median'] = X[['Horizontal_Distance_To_Hydrology',
                                'Horizontal_Distance_To_Roadways']
                               ].median(axis='columns')
    
    X['Road_Fire_sum'] = X[['Horizontal_Distance_To_Roadways',
                            'Horizontal_Distance_To_Fire_Points']
                           ].sum(axis='columns')

    X['Road_Fire_diff'] = X[['Horizontal_Distance_To_Roadways',
                             'Horizontal_Distance_To_Fire_Points']
                            ].diff(axis='columns').iloc[:, [1]].abs()

    X['Road_Fire_mean'] = X[['Horizontal_Distance_To_Roadways',
                             'Horizontal_Distance_To_Fire_Points']
                            ].mean(axis='columns')

    X['Road_Fire_median'] = X[['Horizontal_Distance_To_Roadways',
                               'Horizontal_Distance_To_Fire_Points']
                              ].median(axis='columns')
    
    X['Hydro_Road_Fire_mean'] = X[['Horizontal_Distance_To_Hydrology',
                                   'Horizontal_Distance_To_Roadways',
                                   'Horizontal_Distance_To_Fire_Points']
                                  ].mean(axis='columns')

    X['Hydro_Road_Fire_median'] = X[['Horizontal_Distance_To_Hydrology',
                                     'Horizontal_Distance_To_Roadways',
                                     'Horizontal_Distance_To_Fire_Points']
                                    ].median(axis='columns')

    X['Hillshade_sum'] = X[['Hillshade_9am',
                            'Hillshade_Noon',
                            'Hillshade_3pm']
                           ].sum(axis='columns')

    X['Hillshade_mean'] = X[['Hillshade_9am',
                             'Hillshade_Noon',
                             'Hillshade_3pm']
                            ].mean(axis='columns')

    X['Hillshade_median'] = X[['Hillshade_9am',
                               'Hillshade_Noon',
                               'Hillshade_3pm']
                              ].median(axis='columns')

    X['Hillshade_min'] = X[['Hillshade_9am',
                            'Hillshade_Noon',
                            'Hillshade_3pm']
                           ].min(axis='columns')

    X['Hillshade_max'] = X[['Hillshade_9am',
                            'Hillshade_Noon',
                            'Hillshade_3pm']
                           ].max(axis='columns')
    
    #X['Elevate_hillshad_noon_dff'] = (X['Hillshade_Noon']-X['Hillshade_Noon'].mean())/X['Hillshade_Noon'].std()-(X['Elevation']-X['Elevation'].mean())/X['Elevation'].std()
    #X['Elevate_hillshad_9am_dff'] = (X['Hillshade_9am']-X['Hillshade_9am'].mean())/X['Hillshade_9am'].std()-(X['Elevation']-X['Elevation'].mean())/X['Elevation'].std()
    #X['Elevate_hillshad_max_dff'] = (X['Hillshade_max']-X['Hillshade_max'].mean())/X['Hillshade_max'].std()-(X['Elevation']-X['Elevation'].mean())/X['Elevation'].std()
    #X['Elevate_hillshad_sum_dff'] = (X['Hillshade_sum']-X['Hillshade_sum'].mean())/X['Hillshade_sum'].std()-(X['Elevation']-X['Elevation'].mean())/X['Elevation'].std()
   
    #X['Hydro_hillshad_noon_dff'] = (X['Hillshade_Noon']-X['Hillshade_Noon'].mean())/X['Hillshade_Noon'].std()-(X['Vertical_Distance_To_Hydrology']-X['Vertical_Distance_To_Hydrology'].mean())/X['Vertical_Distance_To_Hydrology'].std()
    #X['Hydro_hillshad_9am_dff'] = (X['Hillshade_9am']-X['Hillshade_9am'].mean())/X['Hillshade_9am'].std()-(X['Vertical_Distance_To_Hydrology']-X['Vertical_Distance_To_Hydrology'].mean())/X['Vertical_Distance_To_Hydrology'].std()
    #X['Hydro_hillshad_max_dff'] = (X['Hillshade_max']-X['Hillshade_max'].mean())/X['Hillshade_max'].std()-(X['Vertical_Distance_To_Hydrology']-X['Vertical_Distance_To_Hydrology'].mean())/X['Vertical_Distance_To_Hydrology'].std()
    #X['Hydro_hillshad_min_dff'] = (X['Hillshade_min']-X['Hillshade_min'].mean())/X['Hillshade_min'].std()-(X['Vertical_Distance_To_Hydrology']-X['Vertical_Distance_To_Hydrology'].mean())/X['Vertical_Distance_To_Hydrology'].std()
    #X['Elevate_hillshad_dff'] = (X['Hillshade_max']-X['Hillshade_min'])
    
    X['Hydro_acrtan'] = np.arctan((X["Vertical_Distance_To_Hydrology"] + 0.001)/(X["Horizontal_Distance_To_Hydrology"] + 0.01))
    
    X['Hydro_eleva_acrtan'] = np.arctan((X["Vertical_Distance_To_Hydrology"] + 0.001)/(X["Elevation"] + 0.01))
    # Compute Soil_Type number from Soil_Type binary columns
    X['Stoneyness'] = sum(i * X['Soil_Type{}'.format(i)] for i in range(1, 41))
    
    # For all 40 Soil_Types, 1=rubbly, 2=stony, 3=very stony, 4=extremely stony, 0=?
    stoneyness = [4, 3, 1, 1, 1, 2, 0, 0, 3, 1, 
                  1, 2, 1, 0, 0, 0, 0, 3, 0, 0, 
                  0, 4, 0, 4, 4, 3, 4, 4, 4, 4, 
                  4, 4, 4, 4, 1, 4, 4, 4, 4, 4]
    
    # Replace Soil_Type number with "stoneyness" value
    X['Stoneyness'] = X['Stoneyness'].replace(range(1, 41), stoneyness)

    return X

In [15]:
X_add_fea = add_features(X)
test_data_add_fea = add_features(test_data)
total_data = pd.concat([X_add_fea,test_data_add_fea])

In [16]:
feats, drop_imp_df = drop_unimportance_features(total_data)
X_drop_fea = drop_imp_df[drop_imp_df.index < 15121]
test_drop_fea = drop_imp_df[drop_imp_df.index >= 15121]

In [17]:
if tune_:
    importance_check(rfc, X_add_fea, y)
    cv_check(rfc, X_add_fea, y, cv = 5)

In [18]:
X_add_fea = drop_unimportance_features_rate(rfc, X_add_fea, y, 0.001)
if tune_:
    cv_check(rfc, X_add_fea, y, cv = 5)

In [19]:
if tune_:
    kde_check(X_add_fea,y,['Elevate_hillshad_min_dff','Elevate_hillshad_max_dff','Elevate_hillshad_9am_dff','Elevate_hillshad_noon_dff'])

In [20]:
if tune_:
    kde_check(X_add_fea,y,['Hydro_hillshad_min_dff','Hydro_hillshad_max_dff','Hydro_hillshad_9am_dff','Hydro_hillshad_noon_dff'])

In [21]:
if tune_:
    kde_check(X_add_fea,y,['Vertical_Distance_To_Hydrology','Hillshade_sum','Hillshade_max'])

In [22]:
if tune_:
    show_feature_correl(X_drop_fea, w=20,h=18)

In [23]:
if tune_:
    _ = importance_check(rfc, X_drop_fea, y)

In [24]:
if tune_:
    cv_check(stacked_model_with_all_best,X_drop_fea, y)

In [25]:
def search_result_check(clf, clf_name):
    cv_keys = ('mean_test_score', 'std_test_score', 'params')

    #for item in enumerate(["a", "b", "c"]):
    #    print item

    #(0, "a")
    #(1, "b")
    #(2, "c")
    for r, _ in enumerate(clf.cv_results_['mean_test_score']):
        print("%s: %0.3f +/- %0.2f %r"
              % (clf_name, clf.cv_results_[cv_keys[0]][r],
                 clf.cv_results_[cv_keys[1]][r] / 2.0,
                 clf.cv_results_[cv_keys[2]][r]))

    print('%s: Best parameters: %s' % (clf_name,clf.best_params_))
    print('%s: Accuracy: %.2f' % (clf_name,clf.best_score_))

In [31]:
# add more models and hopefully the result should be better
rfc = RandomForestClassifier(n_estimators = 300, n_jobs=-1) 
etc = ExtraTreesClassifier(n_estimators = 300,n_jobs=-1) 
adac = AdaBoostClassifier(n_estimators = 200)
lgb = LGBMClassifier(n_estimators = 1000,n_jobs=-1)
#xgb = XGBClassifier(n_estimators = 200) xgboost is not applicable in stacked model
catb_g =  CatBoostClassifier(n_estimators = 1000, task_type='GPU', devices='0:0')

models = {"rfc":rfc, 'etc':etc,'adac':adac, 'lgb':lgb, 'catb':catb_g}

#catb_g.fit(X_drop_fea,y)


#For the GPU usuage of catboost, it will speedup the process dramatically, however, there were problem
#first is the CUDA version is too low 8.9 which is lowever than the catboost needed version 9.1+, upgrade the cuda version
# seconds is the memor already is not enough for catboost if the n_jobs is more than one, reduce the n_jobs fix the issue


In [ ]:
paremeters_check={
    "n_estimators":[400, 500,1000]
}

models = {'catb':catb_g}
for model in models.items():
    search_model = GridSearchCV(estimator=model[1], 
                            param_grid=paremeters_check, 
                            cv=5,
                            verbose=0,
                            scoring='accuracy',
                            refit=True)
    search_model.fit(X_drop_fea,y)
    search_result_check(search_model,model[0])

In [32]:
## if I set n_jobs here, there will always be some error for the catb_g with memory is not enough. Alternatively, I set
# the n_jobs in each classifier individually, and check if its working.
stacked_model_more_models = StackingCVClassifier(classifiers=[rfc,etc,lgb,catb_g],
                             meta_classifier=etc,
                             cv=5,
                             use_probas=True,
                             drop_last_proba=False,
                      
                                                 use_features_in_secondary=True,
                             verbose=1,
                             store_train_meta_features=True,
                             #n_jobs=-1,
                             random_state=0)
stacked_model_more_models.fit(X_drop_fea, y)

Fitting 4 classifiers...
Fitting classifier1: randomforestclassifier (1/4)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   27.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: extratreesclassifier (2/4)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   12.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier3: lgbmclassifier (3/4)


...

500:	learn: 0.4468678	total: 5.73s	remaining: 5.71s
501:	learn: 0.4465835	total: 5.74s	remaining: 5.69s
502:	learn: 0.4461018	total: 5.75s	remaining: 5.68s
503:	learn: 0.4458471	total: 5.76s	remaining: 5.67s
504:	learn: 0.4453800	total: 5.77s	remaining: 5.66s
505:	learn: 0.4450251	total: 5.78s	remaining: 5.65s
506:	learn: 0.4446956	total: 5.79s	remaining: 5.63s
507:	learn: 0.4443817	total: 5.81s	remaining: 5.62s
508:	learn: 0.4441721	total: 5.82s	remaining: 5.61s
509:	learn: 0.4438751	total: 5.83s	remaining: 5.6s
510:	learn: 0.4435970	total: 5.84s	remaining: 5.59s
511:	learn: 0.4431512	total: 5.85s	remaining: 5.58s
512:	learn: 0.4429500	total: 5.86s	remaining: 5.56s
513:	learn: 0.4427000	total: 5.87s	remaining: 5.55s
514:	learn: 0.4424315	total: 5.88s	remaining: 5.54s
515:	learn: 0.4420728	total: 5.89s	remaining: 5.53s
516:	learn: 0.4417980	total: 5.9s	remaining: 5.51s
517:	learn: 0.4414380	total: 5.92s	remaining: 5.5s
518:	learn: 0.4411131	total: 5.93s	remaining: 5.49s
519:	learn: 0.4

667:	learn: 0.4030306	total: 7.57s	remaining: 3.76s
668:	learn: 0.4028802	total: 7.58s	remaining: 3.75s
669:	learn: 0.4027228	total: 7.59s	remaining: 3.74s
670:	learn: 0.4024483	total: 7.6s	remaining: 3.73s
671:	learn: 0.4022749	total: 7.61s	remaining: 3.71s
672:	learn: 0.4020790	total: 7.62s	remaining: 3.7s
673:	learn: 0.4018824	total: 7.63s	remaining: 3.69s
674:	learn: 0.4016084	total: 7.64s	remaining: 3.68s
675:	learn: 0.4012321	total: 7.66s	remaining: 3.67s
676:	learn: 0.4010669	total: 7.67s	remaining: 3.66s
677:	learn: 0.4009275	total: 7.68s	remaining: 3.65s
678:	learn: 0.4006835	total: 7.69s	remaining: 3.63s
679:	learn: 0.4004574	total: 7.7s	remaining: 3.62s
680:	learn: 0.4002206	total: 7.71s	remaining: 3.61s
681:	learn: 0.4000012	total: 7.72s	remaining: 3.6s
682:	learn: 0.3997774	total: 7.73s	remaining: 3.59s
683:	learn: 0.3996727	total: 7.74s	remaining: 3.58s
684:	learn: 0.3994938	total: 7.75s	remaining: 3.57s
685:	learn: 0.3993263	total: 7.76s	remaining: 3.55s
686:	learn: 0.39

832:	learn: 0.3690164	total: 9.4s	remaining: 1.88s
833:	learn: 0.3688295	total: 9.41s	remaining: 1.87s
834:	learn: 0.3685974	total: 9.42s	remaining: 1.86s
835:	learn: 0.3684024	total: 9.43s	remaining: 1.85s
836:	learn: 0.3681804	total: 9.44s	remaining: 1.84s
837:	learn: 0.3679914	total: 9.45s	remaining: 1.83s
838:	learn: 0.3677723	total: 9.46s	remaining: 1.82s
839:	learn: 0.3676249	total: 9.48s	remaining: 1.8s
840:	learn: 0.3673176	total: 9.49s	remaining: 1.79s
841:	learn: 0.3672310	total: 9.5s	remaining: 1.78s
842:	learn: 0.3670186	total: 9.51s	remaining: 1.77s
843:	learn: 0.3668232	total: 9.52s	remaining: 1.76s
844:	learn: 0.3666438	total: 9.53s	remaining: 1.75s
845:	learn: 0.3665172	total: 9.54s	remaining: 1.74s
846:	learn: 0.3663295	total: 9.55s	remaining: 1.73s
847:	learn: 0.3662365	total: 9.56s	remaining: 1.71s
848:	learn: 0.3660663	total: 9.57s	remaining: 1.7s
849:	learn: 0.3659628	total: 9.59s	remaining: 1.69s
850:	learn: 0.3658086	total: 9.6s	remaining: 1.68s
851:	learn: 0.365

998:	learn: 0.3416819	total: 11.2s	remaining: 11.2ms
999:	learn: 0.3415334	total: 11.2s	remaining: 0us


StackingCVClassifier(classifiers=[RandomForestClassifier(bootstrap=True,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         n_estimators=300,
                                                         n_jobs=

In [ ]:
if tune_
    cv_check(stacked_model_more_models,X_drop_fea, y)

In [33]:
save_result(stacked_model_more_models, test_drop_fea)

In [ ]:
# for the linear aggregation, the normalize has some performance improvement
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial')
cv_check(lr, X_drop_fea, y)

# try to normalize the samples and check again for the lr
norm_scaler = StandardScaler()
norm_scaler.fit(X_drop_fea)
norm_X_drop_fea=norm_scaler.transform(X_drop_fea)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial')
cv_check(lr, norm_X_drop_fea, y)

In [ ]:
# can check this directliy in stacking models, however, too many times
stacked_model_more_models.get_params().keys()
# parameter check using 

paremeters_check={
    "meta_classifier__n_estimators":[100, 200, 300, 400],
    "lgbmclassifier__n_estimators":[50, 100, 200, 300, 400],
    "randomforestclassifier__n_estimators":[50, 100, 200, 300, 400],
    "catboostclassifier__n_estimators":[50, 100, 200, 300, 400],
    "extratreesclassifier__n_estimators":[50, 100, 200, 300, 400]

}

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
search_model = GridSearchCV(estimator=stacked_model_more_models, 
                    param_grid=paremeters_check, 
                    cv=5,
                    verbose=1,
                    scoring='accuracy',
                    refit=True)
search_model.fit(X_drop_fea,y)
search_result_check(search_model)